In [1]:
from ollama import chat
from ollama import ChatResponse
from pydantic import BaseModel
import subprocess
import gc
import pandas as pd
import json
from icecream import ic
import os
from tqdm import tqdm
import time

In [2]:
# Ones installed on my pc
model_names = [
    "llama3.1",
    # "olmo-3:7b-instruct",
    # "olmo-3",        # Cannot disable the thinking here but it's essentially the preivous one
    # "granite3.3",
    # "ministral-3",
    # "qwen3",
    # "qwen2.5-coder",
    # "deepseek-r1",   # here with no thinking
    # "deepseek-r1",
    # "gemma3",
    # "phi4-mini",
]

model_thinking = [
    False,
    # False,
    # True,
    # False,
    # False,
    # False,
    # False,
    # True,
    # False,
    # False,
    # False
]

In [3]:
SYSTEM_PROMPT = (
    """
    You are an expert NLU annotator. Your job is to rate how plausible a candidate meaning (sense)
    is for the HOMONYM used in the target sentence within the short story.

    Return ONLY a single JSON object with one key: "score" and an integer value 1, 2, 3, 4 or 5.
    Integer mapping:
      1 = Definitely not
      2 = Probably not
      3 = Ambiguous / Unsure
      4 = Probably yes
      5 = Definitely yes

    The response must be a JSON object and nothing else, for example: {{"score": 4}}
    """
)

USER_PROMPT = (
    """
    [STORY]
    {full_story_text}

    [HOMONYM]
    {homonym}

    [CANDIDATE SENSE]
    {sense_text}

    [ADDITIONAL CONTEXT]
    A DeBERTa model (Accuracy: {deberta_accuracy:.2f}, Spearman Correlation: {deberta_spearman:.2f}) predicted a score of {deberta_prediction:.2f} for this example.
    You can use this information to guide your decision, but rely on your own judgment if the context strongly suggests otherwise.

    [TASK]
    Based on the STORY above, decide how plausible it is that the HOMONYM is used with the
    CANDIDATE SENSE in the target sentence.

    Return ONLY a single JSON object with one key "score" and an integer value (1-5)
    as described by the system message. Example output: {{"score": 3}}
    """
)

In [4]:
def create_full_story_text(item):
    """Compose the story text used as context for rating.

    Uses `precontext`, `sentence`, and `ending` fields when available and joins them into a single string.
    """
    fullstory = f"{item.get('precontext', '')} {item.get('sentence', '')} {item.get('ending', '')}"
    return fullstory.strip()


def create_message(item, deberta_pred, deberta_scores):
    sense = f"{item.get('judged_meaning', '')} as in \"{item.get('example_sentence', '')}\"".strip()
    homonym = item.get("homonym", "")
    full_story_text = create_full_story_text(item)

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT.format(
            full_story_text=full_story_text,
            homonym=homonym,
            sense_text=sense,
            deberta_accuracy=deberta_scores['accuracy'],
            deberta_spearman=deberta_scores['spearman'],
            deberta_prediction=deberta_pred
        )},
    ]
    return messages

In [5]:
TRAIN_JSON_FILE = "../data/train.json"
DEV_JSON_FILE = "../data/dev.json"
DEBERTA_PRED_FILE = "../deberta-finetune-2/predictions.jsonl"
DEBERTA_SCORE_FILE = "../deberta-finetune-2/score.json"

def load_data(file_path):
    """
    Loads the json containing the dataset and return a pandas dataframe.
    """
    with open(file_path, 'r') as f:
        data = json.load(f)
    # Transpose because the json is {id: {features...}, ...}
    df = pd.DataFrame(data).T
    # Ensure 'average' is float
    df['average'] = df['average'].astype(float)
    # Ensure 'choices' is list (for scoring later)
    return df

def load_deberta_results(pred_file, score_file):
    with open(score_file, 'r') as f:
        scores = json.load(f)
    
    preds = {}
    with open(pred_file, 'r') as f:
        for line in f:
            item = json.loads(line)
            preds[str(item['id'])] = item['prediction']
            
    return scores, preds

df_train = load_data(TRAIN_JSON_FILE)
df_dev = load_data(DEV_JSON_FILE)
deberta_scores, deberta_preds = load_deberta_results(DEBERTA_PRED_FILE, DEBERTA_SCORE_FILE)

In [6]:
class Score(BaseModel):
    score: int

In [7]:
# Random element in the dataset
sample = df_dev.sample(1)
idx = str(sample.index[0])
item = sample.iloc[0].to_dict()

# Get deberta prediction
deberta_pred = deberta_preds.get(idx, 0.0)

messages = create_message(item, deberta_pred, deberta_scores)

model_number = 6  # change to try different models

response: ChatResponse = chat(model="qwen2.5-coder",
                              messages=messages,
                              think=False,
                              format=Score.model_json_schema(),
                              options={
                                  "temperature": 0
                              }
                              )

# ic(messages)
ic(response.model)
ic(response.total_duration * 10e-9)  # convert from ns to s
ic(response.message.role)
ic(response.message.content)
ic(response.message.thinking)
ic(messages)

ic| response.model: 'qwen2.5-coder'
ic| response.total_duration * 10e-9: 153.73313129000002
ic| response.message.role: 'assistant'
ic| response.message.content: '{"score": 2}'
ic| response.message.thinking: None
ic| messages: [{'content': '
              '
                           '    You are an expert NLU annotator. Your job is to rate how '
                           'plausible a candidate meaning (sense)
              '
                           '    is for the HOMONYM used in the target sentence within the '
                           'short story.
              '
                           '
              '
                           '    Return ONLY a single JSON object with one key: "score" and '
                           'an integer value 1, 2, 3, 4 or 5.
              '
                           '    Integer mapping:
              '
                           '      1 = Definitely not
              '
                           '      2 = Probably not
              '
    

[{'role': 'system',
  'content': '\n    You are an expert NLU annotator. Your job is to rate how plausible a candidate meaning (sense)\n    is for the HOMONYM used in the target sentence within the short story.\n\n    Return ONLY a single JSON object with one key: "score" and an integer value 1, 2, 3, 4 or 5.\n    Integer mapping:\n      1 = Definitely not\n      2 = Probably not\n      3 = Ambiguous / Unsure\n      4 = Probably yes\n      5 = Definitely yes\n\n    The response must be a JSON object and nothing else, for example: {{"score": 4}}\n    '},
 {'role': 'user',
  'content': '\n    [STORY]\n    James walked into the room, his eyes scanning the space carefully. On the table in the center lay a large, old book, surrounded by scattered papers. He approached it slowly, feeling a sense of anticipation. There was a good pile on it. He ran his hands across it, enjoying the feeling.\n\n    [HOMONYM]\n    pile\n\n    [CANDIDATE SENSE]\n    a large sum of money (especially as pay or pro

In [8]:
def get_dev_predictions(model_name, df, deberta_preds, deberta_scores, max_examples=None, think=False):
    preds = []
    failed_ids = []
    ids = list(df.index.astype(str))
    if max_examples is not None:
        ids = ids[:max_examples]

    total_start = time.perf_counter()
    per_item_times = []

    for idx in tqdm(ids):
        item = df.loc[idx].to_dict()
        
        deberta_pred = deberta_preds.get(str(idx))
        if deberta_pred is None:
            print(f"Warning: No DeBERTa prediction for id {idx}")
            deberta_pred = 0.0

        messages = create_message(item, deberta_pred, deberta_scores)

        start = time.perf_counter()
        try:
            response: ChatResponse = chat(model=model_name,
                                          messages=messages,
                                          think=think,
                                          format=Score.model_json_schema(),
                                          options={
                                              "temperature": 0
                                          }
                                          )
            elapsed = time.perf_counter() - start

            per_item_times.append((idx, elapsed))

            content = response.message.content
            try:
                s = Score.model_validate_json(content)
                pred = int(s.score)
                if pred < 1 or pred > 5:
                    raise ValueError("score out of range")
            except Exception:
                # Keep id of failed element so it can be removed from evaluation
                print("Invalid JSON or missing/invalid score for item id:", idx, "content:", content)
                pred = None
                failed_ids.append(str(idx))

        except Exception as e:
            elapsed = time.perf_counter() - start
            print(f"Error calling model {model_name} for id {idx}: {e}")
            pred = None
            failed_ids.append(str(idx))
            per_item_times.append((idx, elapsed))

        preds.append({"id": str(idx), "prediction": pred, "time": elapsed})

    total_elapsed = time.perf_counter() - total_start
    # attach summary timings as metadata and failed ids
    return {
        "predictions": preds,
        "failed_ids": failed_ids,
        "total_time": total_elapsed,
        "per_item_times": per_item_times,
        "avg_time": sum(t for _, t in per_item_times) / len(per_item_times) if per_item_times else 0,
    }

In [9]:
for model_name, think in zip(model_names, model_thinking):
    if think:
        MAX_EXAMPLES = 100  # set to an int to limit samples per model
    else:
        MAX_EXAMPLES = None  # set to None for not thinking models

    print(f"\n=== Running model: {model_name} ===")

    # If deepseek check if thinking to have different directoies
    if think:
        OUT_DIR = f"../llm-ollama/zero-shot-deberta/{model_name.replace(':', '-')}-think"
        os.makedirs(OUT_DIR, exist_ok=True)
    else:
        OUT_DIR = f"../llm-ollama/zero-shot-deberta/{model_name.replace(':', '-')}"
        os.makedirs(OUT_DIR, exist_ok=True)

    # get predictions (may take a while if MAX_EXAMPLES is None)
    # res = get_dev_predictions(model_name, df_dev, max_examples=MAX_EXAMPLES)
    res = get_dev_predictions(model_name, df_dev, deberta_preds, deberta_scores, max_examples=MAX_EXAMPLES, think=think)

    preds = res["predictions"]

    pred_file = os.path.join(OUT_DIR, "predictions.jsonl")
    # Only write successful predictions (skip None) so pred/ref sizes are aligned
    with open(pred_file, "w") as f:
        for p in preds:
            if p["prediction"] is None:
                # skip failed predictions (they are recorded in failed_ids)
                continue
            f.write(json.dumps({"id": p["id"], "prediction": p["prediction"]}) + "\n")

    # Save failed ids so they can be excluded from scoring
    failed_file = os.path.join(OUT_DIR, "failed_ids.jsonl")
    with open(failed_file, "w") as f:
        for fid in res.get("failed_ids", []):
            f.write(json.dumps({"id": fid}) + "\n")

    # Save timing info and per-item times
    timing_file = os.path.join(OUT_DIR, "timing.txt")
    with open(timing_file, "w") as f:
        f.write(f"total_time_sec: {res['total_time']:.4f}\n")
        f.write(f"avg_time_sec: {res['avg_time']:.4f}\n")
        f.write("per_item_times_sec:\n")
        for idx, t in res["per_item_times"]:
            f.write(f"{idx}: {t:.4f}\n")

    # Create ref.jsonl from df_dev (respect MAX_EXAMPLES) inside the model folder
    # Exclude any ids that failed JSON parsing so they won't be evaluated
    failed_set = set(res.get("failed_ids", []))
    ref_file = os.path.join(OUT_DIR, "ref.jsonl")
    with open(ref_file, "w") as f:
        for idx, row in df_dev.iterrows():
            if MAX_EXAMPLES is not None and int(idx) >= MAX_EXAMPLES:
                break
            if str(idx) in failed_set:
                # skip items that produced invalid JSON for this model
                continue
            f.write(json.dumps({"id": str(idx), "label": row["choices"]}) + "\n")

    print(f"Predictions saved to {pred_file}")
    print(f"Gold data saved to {ref_file}")
    print(f"Timing saved to {timing_file}")

    # Sanity check: warn if counts differ
    n_preds = sum(1 for _ in open(pred_file, "r"))
    n_refs = sum(1 for _ in open(ref_file, "r"))
    if n_preds != n_refs:
        print(f"Warning: #preds ({n_preds}) != #refs ({n_refs}). failed_ids_len={len(res.get('failed_ids', []))}")

    # If there is failed attemp rewrite all of the ids so that they are consecutive. This is needed for the scoring script
    if len(res.get('failed_ids', [])) > 0:
        # Rewrite pred_file with consecutive ids
        new_pred_file = os.path.join(OUT_DIR, "predictions_consecutive_ids.jsonl")
        with open(pred_file, "r") as fin, open(new_pred_file, "w") as fout:
            for new_id, line in enumerate(fin):
                obj = json.loads(line)
                obj["id"] = str(new_id)
                fout.write(json.dumps(obj) + "\n")
        pred_file = new_pred_file

        # Rewrite ref_file with consecutive ids
        new_ref_file = os.path.join(OUT_DIR, "ref_consecutive_ids.jsonl")
        with open(ref_file, "r") as fin, open(new_ref_file, "w") as fout:
            for new_id, line in enumerate(fin):
                obj = json.loads(line)
                obj["id"] = str(new_id)
                fout.write(json.dumps(obj) + "\n")
        ref_file = new_ref_file

    # Run scoring script for this model outputs
    res = subprocess.run(["python", "../score/scoring.py", ref_file, pred_file, os.path.join(OUT_DIR, "score.json")], capture_output=True, text=True)
    print(res.stdout)
    if res.stderr:
        print("Scoring STDERR:")
        print(res.stderr)

    # 
    subprocess.run(["ollama", "stop", model_name], check=False)
    gc.collect()


=== Running model: llama3.1 ===


100%|██████████| 588/588 [04:47<00:00,  2.04it/s]


Predictions saved to ../llm-ollama/zero-shot-deberta/llama3.1/predictions.jsonl
Gold data saved to ../llm-ollama/zero-shot-deberta/llama3.1/ref.jsonl
Timing saved to ../llm-ollama/zero-shot-deberta/llama3.1/timing.txt
Importing...
Starting Scoring script...
Everything looks OK. Evaluating file ../llm-ollama/zero-shot-deberta/llama3.1/predictions.jsonl on ../llm-ollama/zero-shot-deberta/llama3.1/ref.jsonl
----------
Spearman Correlation: 0.5124590139763266
Spearman p-Value: 1.0981397336641898e-40
----------
Accuracy: 0.7295918367346939 (429/588)
Results dumped into scores.json successfully.

